In [ ]:
"""This notebook is for predicting postpartum depression from Shannon entropy.
The results from this notebook are presented in Table 7 in the paper"""
###import packages
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import (f_classif,SelectKBest)
import scipy.stats as stats
from scipy.cluster import hierarchy
import warnings
###import libraries for log-transformation and imputation
import sys
sys.path.insert(0, '/libraries')
from libraries import Transformation_library
###fix random seed for reproducibility 
import os
os.environ["PYTHONHASHSEED"] = "0"

In [ ]:
###user-defined parameters
is_balanced = 0 #do we want to use weights during classification? (as discussed in section 4.1)
is_median = 1 #do we want to use median or mean when select optimal n (number of features, as discussed in section 4.1)

In [ ]:
###standard parameters
level = 0.5 #balanced accuracy when predictions are made by random guess
test = 0.2 #fraction of data used for test set
val = 0.2 #fraction for validation (from the rest part)
ITER = 5 #number of iteration for random forest (section 3.5)
n_features = 2 #the maximum number of features when selecting by ANOVA (as discussed in section 3.2)

In [ ]:
test_sets = int(1/test) #number of testing sets (5)
val_sets = int(1/val) #number of validation sets (5)
binary = 11 #threshold to be classified as helthy/depressed 
if is_balanced == 1:
    class_weight = "balanced"
else:
    class_weight = None

In [ ]:
#read metadata file with id of participants and time points (depending on dataset number)
Metadata=pd.read_csv("BASIC_metadata_full.csv",sep=',',low_memory=False) #read file with depression levels and ids of participants
Metadata.loc[Metadata.TimePoint=="Trimester2","TimePoint"] = 0 #timepoits are 0,1,2
Metadata.loc[Metadata.TimePoint=="Trimester3","TimePoint"] = 1
Metadata.loc[Metadata.TimePoint=="PostpartumWeek6","TimePoint"] = 2
i = Metadata[Metadata.ReadsNumber < 500000].index #remove insufficient reads
Metadata.loc[i, 'ReadsNumber'] = np.nan
EPDS = 2*np.ones_like(Metadata.EPDS) #2 is for missingness
EPDS[Metadata.EPDS>binary] = 1 #binary labels for depression
EPDS[Metadata.EPDS<=binary] = 0

In [ ]:
###data
profile =pd.read_csv("Species_Profile_full.csv",sep=',',low_memory=False) #read file with compositional data
species=profile.to_numpy()[:,1:]/100 #normilized to 1
ID = profile.Sample_id.to_numpy() #id of a sample in profile

In [ ]:
X_full, labels = [[] for _ in range(2)] #X is 2 values of diversity index, labels are EPDS at tp 2
Individual_ID = Metadata.Individual_ID #id of a person
for i in np.unique(Individual_ID):
    TP = Metadata.TimePoint[Individual_ID == i].to_numpy()
    if sum((TP - np.array([0,1,2]))**2)!=0:
        warnings.warn("TP are missing or in incorrect order") #does not happen in the dataset we have
    Outcomes = EPDS[Individual_ID == i]
    if Outcomes[2]!=2:
        Reads = Metadata.ReadsNumber[Individual_ID == i].to_numpy()
        if 1-np.isnan(Reads[0]) and 1-np.isnan(Reads[1]):
            labels = np.append(labels, Outcomes[2])
            Sample_ID = Metadata.Sample_ID[Individual_ID == i]  # id of a sample in metadata
            Dataset_line = np.zeros((1,2))
            for j in range(2):
                tp_id = Sample_ID[TP == j].to_numpy()
                prob_full = species[np.where(ID == tp_id)[0], :]
                prob = prob_full[prob_full>0]
                Dataset_line[0, j] = -sum(list(map(lambda x: x*np.log(x), prob)))
            X_full = Dataset_line if np.size(X_full) == 0 else np.concatenate([X_full, Dataset_line])
X_f, X_t = X_full[labels == 0], X_full[labels == 1] #f = healthy, t = depressed
size_f, size_t = np.shape(X_f)[0], np.shape(X_t)[0]
print("sizes of complete sets:",size_f,size_t)
test_f, test_t = int(size_f*test), int(size_t*test) # sizes of test sets
AUC_TEST, N_OPT, SENSITIVITY, SPECIFICITY = [[] for _ in range(4)]
aucs_val = np.zeros((val_sets, ITER, n_features))
for TS in range(test_sets): #iterating over test sets
    print("test group", TS)
    test_f_range, test_t_range = range(TS * test_f, (TS + 1) * test_f), range(TS * test_t, (TS + 1) * test_t)
    rest_f_range, rest_t_range = np.setdiff1d(range(size_f), test_f_range), np.setdiff1d(range(size_t), test_t_range)
    X_f_test, X_t_test = X_f[test_f_range], X_t[test_t_range]
    X_test = np.concatenate([X_f_test,X_t_test])
    label_test = np.zeros(test_f + test_t)
    label_test[test_f:] = 1
    rest_f, rest_t = np.size(rest_f_range), np.size(rest_t_range)
    val_f, val_t = int(rest_f * val), int(rest_t * val) #sizes of validation sets
    train_f, train_t = rest_f - val_f, rest_t - val_t #the rest of rest is for training set
    print("sizes of train sets", train_f, train_t)
    for VS in range(val_sets): #iterating over vaidation sets
        print("val group", VS)
        val_f_range, val_t_range = rest_f_range[range(VS* val_f, (VS + 1) * val_f)], rest_t_range[range(VS* val_t, (VS + 1) * val_t)]
        train_f_range, train_t_range = np.setdiff1d(rest_f_range, val_f_range), np.setdiff1d(rest_t_range, val_t_range)
        X_f_val, X_t_val, X_f_train, X_t_train = X_f[val_f_range], X_t[val_t_range], X_f[train_f_range], X_t[train_t_range]
        X_val, X_train = np.concatenate([X_f_val, X_t_val]), np.concatenate([X_f_train, X_t_train])
        label_val, label_train = np.zeros(val_f + val_t), np.zeros(train_f + train_t)
        label_val[val_f:], label_train[train_f:] = 1, 1
        stats_train, stats_val = X_train, X_val
        n_features = np.min([n_features, np.shape(stats_train)[1]])
        for j in range(n_features):  #choose the best j + 1 features using ANOVA (section 3.2)
            sel = SelectKBest(f_classif, k=j + 1).fit(pd.DataFrame(stats_train), label_train)
            Index = sel.get_support()
            for seed in range(ITER): #classify by random forest with different random seeds (section 3.4)
                rf = RandomForestClassifier(random_state = seed, class_weight=class_weight).fit(stats_train[:,Index], label_train)
                y_pred = rf.predict(stats_val[:,Index])
                aucs_val[VS, seed, j] = roc_auc_score(label_val, y_pred)
    if is_median == 1: #optimal number of features by median of mean
        print("medians", np.median(aucs_val, axis=(0, 1))) #average by validation sets and random seeds
        n_opt = np.argmax(np.median(aucs_val, axis=(0, 1))) + 1
    else:
        print("means", np.mean(aucs_val, axis = (0,1)))
        n_opt=np.argmax(np.mean(aucs_val, axis = (0,1))) + 1
    N_OPT =np.append(N_OPT, n_opt)
    X_rest, label_rest = np.concatenate([X_train, X_val]), np.concatenate([label_train,label_val])#concatenate everything except testing set
    stats_rest, stats_test = X_rest, X_test
    sel = SelectKBest(f_classif, k=n_opt).fit(pd.DataFrame(stats_rest), label_rest) #select optimal number of features
    Index = sel.get_support()
    print("test features:",sel.get_feature_names_out()) #from features namas we can reconstruct what bacteria we use
    for seed in range(ITER): #iterations for random forest on test sets
        rf = RandomForestClassifier(random_state = seed, class_weight=class_weight).fit(stats_rest[:, Index], label_rest)
        print("importances",rf.feature_importances_) #display importances of features
        y_pred = rf.predict(stats_test[:, Index])
        print("auc", roc_auc_score(label_test, y_pred)) #display balanced accuracy
        AUC_TEST = np.append(AUC_TEST, roc_auc_score(label_test, y_pred)) # balanced accuracy on testing set
        SENSITIVITY = np.append(SENSITIVITY, sum(y_pred[label_test==1]==1)/sum(label_test==1))
        SPECIFICITY = np.append(SPECIFICITY, sum(y_pred[label_test==0]==0)/sum(label_test==0))

In [ ]:
#prining the results for tables
np.save(f"BA_Entropy",AUC_TEST)
np.save(f"sens_Entropy",SENSITIVITY)
np.save(f"spec_Entropy",SPECIFICITY)
Q_bar, STD = np.mean(AUC_TEST), np.std(AUC_TEST)
z = 1.96
result= stats.ttest_1samp(AUC_TEST, level, alternative="greater")
print("mean value of accuracy:", np.round(Q_bar,2))
print("standard deviation of accuracy:", np.round(STD,2))
print("Confidence interval:", np.round(Q_bar-z*STD,2), np.round(Q_bar+z*STD,2))
print("p value",result.pvalue)
print("Sensitivity:",np.round(np.mean(SENSITIVITY),2))
print("Specificity:",np.round(np.mean(SPECIFICITY),2))
print("optimal number of features for classification:", N_OPT)
print("mean value of optimal numbers:",np.mean(N_OPT))